In [7]:
import lightgbm as lgb
import pandas as pd
import numpy as np

In [8]:
X = pd.read_csv('X_ef.csv', index_col='id')
y = pd.read_csv('ef_Y.csv',header=None, index_col=0)
y = y[1]
print(X.shape)
print(y.shape)

(1874, 301)
(1874,)


In [9]:
cat_cols = []
cols = X.columns
for col in cols:
    if col == 'cl_EF_射血分数':
        continue
    if X[col].dtype == 'O':
        cat_cols.append(col)
        X[col] = X[col].astype('category')
        X[col] = pd.Categorical(X[col].cat.codes+1)

In [46]:
## Cross validation to search for good parameters
lr = 0.02
for num_l in [13]:
    for depth in [12]:
        lgb_params = {
     #'num_boost_round': 2500,
    'learning_rate':lr,
    'max_depth': depth,
    'num_leaves': num_l, 
    'objective': 'regression',
    'num_class':1,
    'tree_learner':'voting',
    'metric':'mean_squared_error',
    'feature_fraction': 0.4,
    'bagging_fraction': 0.6,
    'min_data_in_leaf': 15,
    'max_bin': 150}
        dtrain_lgb = lgb.Dataset(X, label=y,categorical_feature=cat_cols)
        cv_result_lgb = lgb.cv(lgb_params, 
                       dtrain_lgb, 
                       num_boost_round = 800, 
                       nfold=10, 
                       stratified=False,
                       early_stopping_rounds=20, 
                       verbose_eval=200, 
                       show_stdv=True)
        #results.append(cv_result_lgb['l2-mean'][-1])
        print('score   learning rate iterations  max_depth  num_leaves')
        print('%.8f   %.4f    %d     %d        %d' 
              %(cv_result_lgb['l2-mean'][-1], lr, len(cv_result_lgb['l2-mean']), depth, num_l))

/home/xps/anaconda3/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[200]	cv_agg's l2: 40.688 + 4.32193
[400]	cv_agg's l2: 39.7815 + 3.85122
score   learning rate iterations  max_depth  num_leaves
39.71922713   0.0200    441     12        13


In [48]:
# Train the model with good parameters
lgb_params = {
     #'num_boost_round': 2500,
    'learning_rate':lr,
    'max_depth': depth,
    'num_leaves': num_l, 
    'objective': 'regression',
    'num_class':1,
    'tree_learner':'voting',
    'metric':'mean_squared_error',
    'feature_fraction': 0.4,
    'bagging_fraction': 0.6,
    'min_data_in_leaf': 15,
    'max_bin': 150}
dtrain_lgb = lgb.Dataset(X, label=y,categorical_feature=cat_cols)
gbm = lgb.train(lgb_params,
                dtrain_lgb,
                num_boost_round = 441
               )

/home/xps/anaconda3/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [104]:
def print_and_save_feature_importance(model, display = True):
    feat_weights = model.feature_importance()
    feat_names = model.feature_name()
    sorted_idx = np.argsort(feat_weights)[::-1]
    df = pd.DataFrame(columns=['feature name','importance'])
    for i,idx in enumerate(sorted_idx):
        if display:
            print('%s: %d' %(feat_names[idx],feat_weights[idx]))
        df.loc[i] = [feat_names[idx],feat_weights[idx]]
    return df

In [108]:
df =  print_and_save_feature_importance(gbm, display=False)
df.to_csv('feature_importance.csv')
df.head(20)

,feature name,importance
0,2404,99
1,2333,99
2,10004,87
3,1106,79
4,cl_BMI_sq,79
5,1815,78
6,191,75
7,10002,75
8,190,74
9,669021,73
